In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from loguru import logger
from datetime import datetime
import glob
from matplotlib import pyplot as plt
from collections import Counter
import time
from fake_useragent import UserAgent
from collections import Counter
import urllib

nest_asyncio.apply()


## Спарсим рамблер

In [ ]:
signs = ['aries', 'gemini', 'taurus', 'cancer', 'leo', 'virgo', 'libra', 'scorpio', 'sagittarius', 'capricorn',
         'aquarius', 'pisces']
data = pd.read_csv('data.csv', names=['date', 'sign', 'text'])

for i in range(2004, 2021):
    for j in range(1, 13):
        for k in range(1, 32):
            for sign in signs:
                try:
                    year = i
                    month = j
                    day = k
                    if month < 10:
                        month = '0' + str(month)
                    if day < 10:
                        day = '0' + str(day)
                    date = '%s-%s-%s' % (year, month, day)
                    url = 'https://horoscopes.rambler.ru/%s/%s/?updated' % (sign, date)  # url для второй страницы
                    print(url)
                    r = requests.get(url)
                    response = r.text.encode('utf-8')

                    soup = BeautifulSoup(response, features="lxml")
                    text = soup.find('div', {'class': '_1dQ3'}).text
                    data = data.append({'date': date, 'sign': sign, 'text': text}, ignore_index=True)
                    data.to_csv('data.csv', encoding='utf-8')
                    print('saved date=', date, ' sign=', sign)
                except Exception:
                    continue

## Сайт Ростова-на-Дону

https://www.1rnd.ru/

In [ ]:
news = []
url = 'https://www.1rnd.ru/news/3089599'
print(url)
r = requests.get(url)
response = r.text
soup = BeautifulSoup(response, 'html.parser')

news_text = soup.findAll('p', {'style': "text-align:justify;"})
date_text = soup.findAll('div', {'class': "article-info__time"})
header_text = soup.findAll('div', {'class': 'title-container inner-title'})

if news_text:
    print('success')
    text_line = ''
    for text in news_text:
        text_line = text_line + text.get_text()
    news.append([header_text[0].get_text(), date_text[0].get_text().split(), text_line])

In [ ]:
news = pd.DataFrame({'header': [0], 'date': [0], 'news': [0]})

In [ ]:
start1 = 7135
start2 = 300000
start3 = 600000
start4 = 900000
start5 = 1200000
start6 = 1500000
start7 = 1800000
start8 = 2100000
start9 = 2400000
start10 = 2700000

end1 = 299999
end2 = 599999
end3 = 899999
end4 = 1199999
end5 = 1499999
end6 = 1799999
end7 = 2099999
end8 = 2399999
end9 = 2699999
end10 = 2999999


async def get_news(start, end):
    global news
    for i in range(start, end):
        url = 'https://www.1rnd.ru/news/{0}'.format(str(i))
        print(url)

        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:

                response = await response.text()
                soup = BeautifulSoup(response, 'html.parser')

                news_text = soup.findAll('p', {'style': "text-align:justify;"})
                date_text = soup.findAll('div', {'class': "article-info__time"})
                header_text = soup.findAll('div', {'class': 'title-container inner-title'})

                if news_text:
                    print('success')
                    text_line = ''
                    for text in news_text:
                        text_line = text_line + text.get_text()

                    date = date_text[0].get_text()
                    header = header_text[0].get_text()

                    news = news.append({'header': header, 'date': date, 'news': text_line}, ignore_index=True)
                    news.to_csv('news.csv', encoding='utf-8')


futures = [get_news(start1, end1),
           get_news(start2, end2),
           get_news(start3, end3),
           get_news(start4, end4),
           get_news(start5, end5),
           get_news(start6, end6),
           get_news(start7, end7),
           get_news(start8, end8),
           get_news(start9, end9),
           get_news(start10, end10)]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))
loop.close()